
# Barbero durmiente

Para tratar el problema se  ha implementado una clase que simula la Barberria con los procesos que se ejecutan dentro de ella.
Se proporciona el esqueleto de la implementación de los procesos para que se marquen las zonas de exclusión mutua, de espera o de sincronazación. 
En el constructor de la Barberia se inicilizan todos los cerrojos, semaforos, condiciones y variables que se necesitan para la implementación.

In [1]:
import threading
import time
import random

class Barberia:
    def __init__(self):
        #mecanismos y variables de control clientes y acceso a la sala
        ####### inicializa cada uno con el tipo y valor oportuno####
        #####puedes cambiar los nombres o añadir nuevos#######

        self.MAX_CLIENTES = 3 

        #controla la entrada en la sala de espera
        self.entrarEnSala = threading.Semaphore(3)
        #controla el número de clientes en la sala
        self.clientesEnSala = 0 
        #garantiza exclusion para modificacion de variables de control de sala o acceso a secciones
        self.exclusionMutuaSala = threading.Lock()
        
        #mecanismos y variables de contrrol del barbero
        ####### inicializa cada uno con el tipo y valor oportuno #######
        #####puedes cambiar los nombres o añadir nuevos#######
        #controla el acesso al sillon de corte, puede poner clientes en espera y ser despertados
        self.sillon = threading.Condition()
        #controla lel estado del barbero
        self.barberoOcupado = False
        #puede poner al barbero a dormir y ser despertado
        self.dormir = threading.Condition()
        

    def actividadBarbero(self):
        while True:
            
            print ("\nBarbero: Clientes en la sala de espera: " + str(self.clientesEnSala))
            
            ### Se consulta en exclusion mutua los clientes que hay en la sala. ###
            with self.exclusionMutuaSala:
                if self.clientesEnSala == 0:
                    ### Si no hay clientes esperando se va a dormir hasta que le despierten. ####
                    print ("Barbero: No hay clientes. Se va a dormir.")
                    with self.dormir:    
                        # Se libera el lock de exclusión mutua de la sala para evutar un bloqueo. Se retoma cuando se despierta.
                        self.exclusionMutuaSala.release()
                        self.dormir.wait()
                        self.exclusionMutuaSala.acquire()

                ###  Si hay clientes, el barbero pasa a estar ocupado. ###
                self.barberoOcupado = True
                print("Barbero: Pasa a estar ocupado, listo para cortar el pelo.")
            
            ### Avisa de que tiene el sillón libre. ###
            with self.sillon:
                self.sillon.notify()
                print("Barbero: Tiene el sillón libre. Esperando que se siente un cliente.")

            time.sleep(2.5) # Tiempo que necesita el barbero para cortar el pelo y limpiar la zona de pelado.
            
            ###  El barbero se libera, dando pie a los clientes a que puedan  ###
            with self.exclusionMutuaSala:
                self.barberoOcupado = False
                print("Barbero: Ya ha pelado. Vuelve a estar libre.")
                    
            
    def actividadCliente(self):

        print(threading.current_thread().name + ": Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio.")
        
        ### Según estado de la sala, entrar o esperar ####
        if not self.entrarEnSala.acquire(timeout=10):  # Espera hasta 10 segundos para entrar en la sala.
            print(f"{threading.current_thread().name} : La sala de espera está ocupada demasado tiempo. Se va tras esperar 10 segundos.")
            return

        ### Consultar en exclusion mutua el estado ddel barbero. Cliente a sala de espera o a pelarse directamente. ###
        with self.exclusionMutuaSala:
            if self.barberoOcupado:
                self.clientesEnSala += 1
                print( threading.current_thread().name + ": Se sienta en la sala de espera: (" + str(self.clientesEnSala) + ")" )
            else:
                with self.dormir:
                    self.dormir.notify()
                    print (threading.current_thread().name + ": Despierta al barbero.")

        ### Cuando el sillón queda libre, se sienta a pelarse y libera un sitio en la sala de espera ###
        with self.sillon:
            self.sillon.wait()
            self.entrarEnSala.release()  
            with self.exclusionMutuaSala:
                if self.clientesEnSala > 0:
                    self.clientesEnSala -= 1 
        
            print (threading.current_thread().name + ": Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(" + str(self.clientesEnSala) + ")")
                
            time.sleep(1.5) # Tiempo que dura un corte de pelo.

            ### El cliente se va pelado y deja el sillón libre. ###
            print (threading.current_thread().name + ": Libera el sillón. Abandona la barbería pelado.")

if __name__=="__main__":

        barberia = Barberia()
        
        # Se crea e inicia el hilo del barbero
        barbero = threading.Thread(target = barberia.actividadBarbero)
        barbero.start()
        
        # Se crea e inician los hilos para varios clientes
        for i in range(20):
            cliente = threading.Thread(target = barberia.actividadCliente, name = f'Cliente {i+1}')
            time.sleep(random.uniform(0.5, 2))  # Simula la llegada de clientes en momentos aleatorios
            cliente.start()




Barbero: Clientes en la sala de espera: 0
Barbero: No hay clientes. Se va a dormir.
Cliente 1: Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio.
Cliente 1: Despierta al barbero.
Barbero: Pasa a estar ocupado, listo para cortar el pelo.
Barbero: Tiene el sillón libre. Esperando que se siente un cliente.
Cliente 1: Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(0)
Cliente 1: Libera el sillón. Abandona la barbería pelado.
Cliente 2: Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio.
Cliente 2: Se sienta en la sala de espera: (1)
Barbero: Ya ha pelado. Vuelve a estar libre.

Barbero: Clientes en la sala de espera: 1
Barbero: Pasa a estar ocupado, listo para cortar el pelo.
Barbero: Tiene el sillón libre. Esperando que se siente un cliente.
Cliente 2: Se sienta en el sillón para pelarse. Libera un sitio de la sala de espera:(0)
Cliente 3: Llega a la puerta de la barbería. Espera en la puerta hasta que haya sitio